In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv, svd, eigh
import pandas as pd
from scipy import stats
from sklearn.decomposition import PCA
import iris

In [45]:
X, y = iris.get()

In [53]:
pca = PCA()
pca.fit(X, y)

PCA(copy=True, n_components=None, whiten=False)

In [56]:
eigh(X.T.dot(X))

(array([  3.53031167e+00,   1.20360193e+01,   3.14103073e+02,
          9.20653060e+03]),
 array([[ 0.32345496,  0.49942378,  0.28583096, -0.75116805],
        [-0.32124324, -0.67502499,  0.54488976, -0.37978837],
        [-0.48077482, -0.05471983, -0.70889874, -0.51315094],
        [ 0.74902286, -0.54029889, -0.34475845, -0.16787934]]))

In [48]:
U, s, V = svd(X)

In [42]:
C = 2
X = np.array([[1, 2], [3, 4], [5, 6]])
W = np.eye(C)
N, D = X.shape

E = np.exp(X.dot(W))
S = np.sum(E, axis = 1)
[[E[i, c] / S[i] for c in range(C)] for i in range(N)]

[[0.2689414213699951, 0.7310585786300049],
 [0.2689414213699951, 0.73105857863000479],
 [0.26894142136999516, 0.7310585786300049]]